In [36]:
import requests
import polars as pl
import json
import os
from dotenv import load_dotenv
from classes import Tour, Round
from msgspec.json import decode

load_dotenv()

True

In [37]:
def call_lichess_broadcasts_api(broadcast_tournament_id):
    headers = {
        "Content-Type": "application/json",
        "X-Api-Key": os.environ.get("LICHESS_API_KEY"),
    }

    lichess_broadcast_endpoint = (
        f"https://lichess.org/api/broadcast/{broadcast_tournament_id}"
    )

    response = requests.get(lichess_broadcast_endpoint, headers=headers)

    # return response

    result_json = response.json()

    return result_json, lichess_broadcast_endpoint

In [38]:
tournament_id = "wEuVhT9c"

In [39]:
tournament_json, api_endpoint = call_lichess_broadcasts_api(tournament_id)

In [32]:
def create_broadcast_base_dataframes(json_data, tournament_id):
    lichess_tour = decode(json.dumps(json_data["tour"]), type=Tour)
    lichess_rounds = decode(json.dumps(json_data["rounds"]), type=list[Round])

    lichess_tour_final, lichess_rounds_final = parse_broadcast_tournament_base(
        lichess_tour, lichess_rounds, tournament_id
    )

    lichess_tour_df = pl.DataFrame(lichess_tour_final)
    lichess_rounds_df = pl.DataFrame(lichess_rounds_final)
    # if len(job_location_df) > 0:
    #     job_location_df = job_location_df.with_columns(
    #         pl.col("placement").cast(pl.Int16)
    #     )

    # return [job_outline_df, job_location_df]

    return lichess_tour_df, lichess_rounds_df

In [33]:
lichess_tour_df, lichess_rounds_df = create_broadcast_base_dataframes(
    tournament_json, tournament_id
)

In [27]:
def parse_broadcast_tournament_base(tour_data, rounds_data, tournament_id):
    tour_json = {}
    tour_json["tournament_id"] = tour_data.id
    tour_json["name"] = tour_data.name
    tour_json["slug"] = tour_data.slug
    tour_json["created_at"] = tour_data.createdAt
    tour_json["tier"] = tour_data.tier
    tour_json["image"] = tour_data.image
    tour_json["url"] = tour_data.url
    tour_json["leaderboard"] = tour_data.leaderboard

    rounds_json = []
    for i, chess_round in enumerate(rounds_data):
        round_json_record = {}
        round_json_record["round_id"] = chess_round.id
        round_json_record["tournament_id"] = tournament_id
        round_json_record["name"] = chess_round.name
        round_json_record["slug"] = chess_round.slug
        round_json_record["created_at"] = chess_round.createdAt
        round_json_record["starts_at"] = chess_round.startsAt
        round_json_record["url"] = chess_round.url
        round_json_record["finished"] = chess_round.finished

        #     try:
        #         job_outline_json_record["address_region"] = (
        #             job_outline.address.postalAddress.addressRegion
        #         )
        #         job_outline_json_record["address_country"] = (
        #             job_outline.address.postalAddress.addressCountry
        #         )
        #         job_outline_json_record["address_locality"] = (
        #             job_outline.address.postalAddress.addressLocality
        #         )
        #     except AttributeError:
        #         job_outline_json_record["address_region"] = None
        #         job_outline_json_record["address_country"] = None
        #         job_outline_json_record["address_locality"] = None

        rounds_json.append(round_json_record)

    #         all_job_locations_json.append(job_location_json_record)

    return tour_json, rounds_json

In [28]:
parse_broadcast_tournament_base(lichess_tour, lichess_rounds, tournament_id)

({'tournament_id': 'wEuVhT9c',
  'name': 'FIDE Candidates 2024 | Open',
  'slug': 'fide-candidates-2024--open',
  'created_at': 1710759962579,
  'tier': 5,
  'image': 'https://image.lichess1.org/display?h=400&op=thumbnail&path=loepare:relay:wEuVhT9c:UzQX0Wlo.jpg&w=800&sig=a02fbde968e0095d83e48a6055c16254ee5aa331',
  'url': 'https://lichess.org/broadcast/fide-candidates-2024--open/wEuVhT9c',
  'leaderboard': True},
 [{'round_id': 'AjqSsU1w',
   'tournament_id': 'wEuVhT9c',
   'name': 'Round 1',
   'slug': 'round-1',
   'created_at': 1710760290779,
   'starts_at': 1712256300000,
   'url': 'https://lichess.org/broadcast/fide-candidates-2024--open/round-1/AjqSsU1w',
   'finished': True},
  {'round_id': 'GenKIJ8A',
   'tournament_id': 'wEuVhT9c',
   'name': 'Round 2',
   'slug': 'round-2',
   'created_at': 1711530504000,
   'starts_at': 1712342700000,
   'url': 'https://lichess.org/broadcast/fide-candidates-2024--open/round-2/GenKIJ8A',
   'finished': True},
  {'round_id': 'xQgaUu2y',
   't

In [35]:
tournament_json["tour"]

{'id': 'wEuVhT9c',
 'name': 'FIDE Candidates 2024 | Open',
 'slug': 'fide-candidates-2024--open',
 'description': 'April 4th - 22nd | 8-player double round-robin | Classical time control',
 'createdAt': 1710759962579,
 'tier': 5,
 'image': 'https://image.lichess1.org/display?h=400&op=thumbnail&path=loepare:relay:wEuVhT9c:UzQX0Wlo.jpg&w=800&sig=a02fbde968e0095d83e48a6055c16254ee5aa331',
 'markup': '<p>The <strong>FIDE Candidates Tournament 2024</strong> takes place from the <strong>4th to the 22nd of April in Toronto, Canada</strong>, deciding the challenger of World Champion Ding Liren in the FIDE World Chess Championship 2024. The tournament is played as a <strong>double round-robin with 14 rounds</strong>.</p>\n<p>Time control is <strong>120 minutes for 40 moves, followed by 30 minutes for the rest of the game, with a 30-second increment from move 41.</strong></p>\n<p>Players cannot draw by agreement before Black\'s 40th move. In the case of a tie for first place after 14 rounds, a p

In [34]:
with open("data.json", "w") as f:
    json.dump(tournament_json, f)